In [58]:
import numpy as np
import random
import requests as rq
import sys
import io
from bs4 import BeautifulSoup
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop
from collections import Counter
from datetime import datetime
import keras
import keras.callbacks
from keras.callbacks import TensorBoard

In [59]:
file = open("baddate.txt", encoding="utf-8")
text = file.read()

print(text[0:300])

1 1 EXT. PORTLAND - NIGHT
Establishing shot of Portland, Oregon. *
2 2 EXT. RICHARDS REVIEW OFFICES - NIGHT
Establishing shot of the offices of the Richardson Review,
located in an old warehouse turned into modern office space.
3 3 INT. ALLISON’S OFFICE - NIGHT *
The office of the head of the Richar


In [60]:
file2 = open("nietzsche.txt", encoding="utf-8")
text2 = file2.read()

In [61]:
text3 = text + text2
chars = sorted(list(set(text3)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

seqlen = 40
step = seqlen
sentences = []
for i in range(40, len(text) - seqlen - 1, step):
    sentences.append(text[i: i + seqlen + 1])
    

x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
        x[i, t, char_indices[char_in]] = 1
        y[i, t, char_indices[char_out]] = 1
        
        
model = Sequential()
model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
model.add(Dense(len(chars), activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy']
)

def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas)                     #


def on_epoch_end(epoch, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - seqlen - 1)
    
    for diversity in [0.5]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + seqlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, seqlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.
            
            preds = model.predict(x_pred, verbose=0)
            next_index = sample(preds[0, -1], diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            
            
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=20,
          callbacks=[print_callback])

for i in range(len(model.layers)-1):
    layer = model.layers[i]
    layer.trainable = False
          
#model.fit(x, y,
          #batch_size=128,
          #epochs=150,
          #callbacks=[print_callback])

C:\Users\syosh\AppData\Local\Temp\ipykernel_23696\144191774.py:13: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
C:\Users\syosh\AppData\Local\Temp\ipykernel_23696\144191774.py:14: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), seqlen, len(chars)), dtype=n

Epoch 1/20
21/21 [==============================] - ETA: 0s - loss: 3.7117 - categorical_crossentropy: 3.7117 - accuracy: 0.1351
----- Generating text after Epoch: 0
----- diversity: 0.5
----- Generating with seed: " it happen however it’s
going to happen."
 it happen however it’s
going to happen.   ne s   f t  taeN d 
  ef t   t 
  e g t  tt t le 
m b
 er  te a ase s A nrdd a e  e  ges ee   
a 
e t ese an e
     
aa te tn  anep eis   ah sen i d e  t
e s  e
oe i n eee e  a  e n ll    eR n  a   e    raeter ee o oa se  Iee gee o    ioe     a e in ot a  e 
Hee  the 
  d e   TaaTse e ee i s 
n te ha e t nr aor  
l aa heioe teo   ae ice  aa  o
21/21 [==============================] - 98s 5s/step - loss: 3.7117 - categorical_crossentropy: 3.7117 - accuracy: 0.1351
Epoch 2/20
21/21 [==============================] - ETA: 0s - loss: 3.2721 - categorical_crossentropy: 3.2721 - accuracy: 0.1770
----- Generating text after Epoch: 1
----- diversity: 0.5
----- Generating with seed: " to Conner and 

In [57]:
print(text2[0:300])

What I am now going to relate is the history of the next two centuries.
I shall describe what will happen, what must necessarily happen:
_the triumph of Nihilism._ This history can be written already; for
necessity itself is at work in bringing it about. This future is
already proclaimed by a hundr


In [63]:
from keras.models import Model
 
for i in range(40, len(text2) - seqlen - 1, step):
    sentences.append(text2[i: i + seqlen + 1])
    
        
model2 = Sequential()
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
model2.add(model)

model2.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
model2.add(Dense(len(chars), activation='softmax'))


model2.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy']
)

def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas)                     #


def on_epoch_end(epoch, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text2) - seqlen - 1)
    
    for diversity in [0.5]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text2[start_index: start_index + seqlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(1200):
            x_pred = np.zeros((1, seqlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.
            
            preds = model2.predict(x_pred, verbose=0)
            next_index = sample(preds[0, -1], diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            
            
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model2.fit(x, y,
          batch_size=128,
          epochs=20,
          callbacks=[print_callback])

Epoch 1/20
21/21 [==============================] - ETA: 0s - loss: 3.7099 - categorical_crossentropy: 3.7099 - accuracy: 0.1329
----- Generating text after Epoch: 0
----- diversity: 0.5
----- Generating with seed: "reat that he actually does
sacrifice som"
reat that he actually does
sacrifice somet  ha   i  t s  tan  ht t c ie s  e  e  .e w n     reRe ao  t wh   erhst  aa ha Rd tse  e a o
 e es n i n  y. e n a i oio t     oaiwe ihe o wt  hks er   e    pt o I   o  se htehg . ae at
ci

t i e t n  hni .      s teag t e a rl i.e
di t e  if  o    n   
 ae  e .  ati e  a, t r   we  wn eo  w  f o atar r  reewaasintheu lycal  ar is e  hahi,
 eie  e h i ere
nnoeiey in i   elae
s ..o  h h d oa tyo cetha it 
ea weani  a wt  t
 a)seeyr    n t tenerhe wae f ?o 
 e a ea at at yaee ae to y   ’ee n 
mee  a bnt 
 ahe ey   rashad ega ee wheral in r r  i hae ta ni ocai w
I a  e     ree e e ce o 
 laor   eh   it reeoh e aat o hh  s y ie ae h H Tent  nreamhy  m  
u    t a a  ih g 
   et   le il. ehe.
a   